Import necessary libraries

In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Concatenate, Input,SeparableConv2D, Conv2DTranspose
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

Define parameters

In [ ]:
IMG_HEIGHT, IMG_WIDTH = 128,128
NUM_IMAGES = 2000
BATCH_SIZE = 32
EPOCHS = 50
IMAGE_DIR = "../datasets/dataset2/face_crop"
MASK_DIR = "../datasets/dataset2/face_crop_segmentation"

Load the dataset

In [ ]:
def load_data(image_dir, mask_dir, img_size=(IMG_HEIGHT, IMG_WIDTH)):

    images, masks = [], []
    image_files = sorted(os.listdir(image_dir))
    mask_files = sorted(os.listdir(mask_dir))
    count = 0

    for img_file, mask_file in zip(image_files, mask_files):
        if(count>=NUM_IMAGES):
            break

        # Read images, resize and convert to grayscale
        img = cv2.imread(os.path.join(image_dir, img_file))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, img_size) / 255.0

        # Read masks, resize and convert to grayscale
        mask = cv2.imread(os.path.join(mask_dir, mask_file), cv2.IMREAD_GRAYSCALE)
        mask = cv2.resize(mask, img_size)
        mask = np.expand_dims(mask, axis=-1)
        mask = mask / 255.0

        images.append(img)
        masks.append(mask)
        count+=1

    return np.array(images), np.array(masks)

# Load dataset
X, Y = load_data(IMAGE_DIR, MASK_DIR)
print(f"Dataset loaded: {X.shape}, {Y.shape}")
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.2, random_state=0)

Define the U-Net model

In [ ]:
def unet(input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)):
    inputs = Input(input_shape)

    # Encoder
    c1 = SeparableConv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    p1 = MaxPooling2D((2, 2))(c1)
    c2 = SeparableConv2D(128, (3, 3), activation='relu', padding='same')(p1)
    p2 = MaxPooling2D((2, 2))(c2)
    c3 = SeparableConv2D(256, (3, 3), activation='relu', padding='same')(p2)
    p3 = MaxPooling2D((2, 2))(c3)
    c4 = SeparableConv2D(512, (3, 3), activation='relu', padding='same')(p3)

    # Decoder
    u5 = Conv2DTranspose(128, (3, 3), strides=(2, 2), padding='same', activation='relu')(c4)
    u5 = Concatenate()([u5, c3])
    c5 = SeparableConv2D(128, (3, 3), activation='relu', padding='same')(u5)
    u6 = Conv2DTranspose(64, (3, 3), strides=(2, 2), padding='same', activation='relu')(c5)
    u6 = Concatenate()([u6, c2])
    c6 = SeparableConv2D(64, (3, 3), activation='relu', padding='same')(u6)
    u7 = Conv2DTranspose(32, (3, 3), strides=(2, 2), padding='same', activation='relu')(c6)
    u7 = Concatenate()([u7, c1])
    c7 = SeparableConv2D(32, (3, 3), activation='relu', padding='same')(u7)

    outputs = Conv2D(1, (1, 1), activation='sigmoid')(c7)

    model = Model(inputs, outputs)
    return model

Define the IoU and Dice metrics, and the Binary CrossEntropy + Dice Loss function

In [ ]:
def iou(y_true, y_pred):
    y_pred = tf.cast(y_pred > 0.5, tf.float32)
    intersection = tf.reduce_sum(y_true * y_pred)
    union = tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) - intersection
    return intersection / (union + 1e-7)

def dice(y_true, y_pred):
    y_pred = tf.cast(y_pred > 0.5, tf.float32)
    intersection = tf.reduce_sum(y_true * y_pred)
    return (2. * intersection) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + 1e-7)

# Define BCE + Dice Loss
def bce_dice_loss(y_true, y_pred):
    return tf.keras.losses.BinaryCrossentropy()(y_true, y_pred) + 1 - dice(y_true, y_pred)

Train the model

In [ ]:
# Compile Model
model = unet()
model.compile(optimizer=Adam(learning_rate=1e-3), loss=bce_dice_loss, metrics=[iou, dice])

# Train Model
history = model.fit(
    X_train, Y_train,
    validation_data=(X_val, Y_val),
    batch_size=BATCH_SIZE,
    epochs=EPOCHS
)

Evaluate it for a single sample

In [ ]:
# Evaluate Model
def predict_sample(model, X_val, Y_val, index=70):
    pred_mask = model.predict(np.expand_dims(X_val[index], axis=0))[0]
    pred_mask = (pred_mask > 0.5).astype(np.uint8)

    plt.figure(figsize=(10, 5))
    plt.subplot(1, 3, 1)
    plt.imshow(X_val[index])
    plt.title("Original Image")
    plt.subplot(1, 3, 2)
    plt.imshow(Y_val[index].squeeze(), cmap="gray")
    plt.title("Ground Truth Mask")
    plt.subplot(1, 3, 3)
    plt.imshow(pred_mask.squeeze(), cmap="gray")
    plt.title("Predicted Mask")
    plt.show()

# Test model on a sample image
predict_sample(model, X_val, Y_val)

Calculate the validation IoU and Dice scores

In [ ]:
def calculate_iou(y_true, y_pred, threshold=0.5):
    if y_true.dtype != np.bool_:
        y_true = y_true.astype(bool)
    if y_pred.dtype != np.bool_:
        y_pred = (y_pred > threshold).astype(bool)
    intersection = np.logical_and(y_true, y_pred).sum()
    union = np.logical_or(y_true, y_pred).sum()
    if union == 0:
        return 1.0
    return intersection / union

def calculate_dice(y_true, y_pred, threshold=0.5):
    if y_true.dtype != np.bool_:
        y_true = y_true.astype(bool)
    if y_pred.dtype != np.bool_:
        y_pred = (y_pred > threshold).astype(bool)
    intersection = np.logical_and(y_true, y_pred).sum()
    total = y_true.sum() + y_pred.sum()
    if total == 0:
        return 1.0
    return 2 * intersection / total

# Predict on validation set
y_predicted = model.predict(X_val)

# Compute IoU for each image
iou_scores = []
dice_scores = []
for i in range(len(Y_val)):
    iou = calculate_iou(Y_val[i], y_predicted[i])
    dice = calculate_dice(Y_val[i], y_predicted[i])
    iou_scores.append(iou)
    dice_scores.append(dice)

mean_iou = np.mean(iou_scores)
mean_dice = np.mean(dice_scores)

print(f"\nMean IoU on Validation Set: {mean_iou:.4f}")
print(f"Mean Dice on Validation Set: {mean_dice:.4f}")